# ASL Model Training - Anti-Overfitting Version

This notebook trains an ASL recognition model with strong regularization to prevent overfitting.

**Improvements:**
- Early stopping
- L2 regularization
- Higher dropout rates
- Better data augmentation
- Larger image size (64x64)
- Validation monitoring

## Cell 1: Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Cell 2: Configuration

In [ ]:
# Training configuration
TRAINING_DATA_DIR = "combined_dataset"
IMAGE_SIZE = 64  # Increased from 32 for better detail
BATCH_SIZE = 32
EPOCHS = 50  # Max epochs (early stopping will prevent overfitting)
LEARNING_RATE = 0.001

print("="*60)
print("ASL Model Training - ANTI-OVERFITTING VERSION")
print("="*60)
print(f"Image Size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Max Epochs: {EPOCHS}")
print(f"Learning Rate: {LEARNING_RATE}")

## Cell 3: Check Dataset

In [ ]:
# Verify dataset exists
if not os.path.exists(TRAINING_DATA_DIR):
    print(f"❌ ERROR: '{TRAINING_DATA_DIR}' folder not found!")
    raise FileNotFoundError(f"Dataset directory not found: {TRAINING_DATA_DIR}")

# Count classes and images
classes = [d for d in os.listdir(TRAINING_DATA_DIR) 
           if os.path.isdir(os.path.join(TRAINING_DATA_DIR, d))]
print(f"✓ Found {len(classes)} classes: {sorted(classes)}")

# Count total images
total_images = sum([len(os.listdir(os.path.join(TRAINING_DATA_DIR, c))) 
                    for c in classes])
print(f"✓ Total images: {total_images:,}")
print(f"✓ Average per class: {total_images // len(classes):,}")

## Cell 4: Data Augmentation Setup

In [ ]:
print("[1/5] Setting up data augmentation...")

# STRONG augmentation for training (prevents overfitting)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Rotate ±20 degrees
    width_shift_range=0.15,  # Horizontal shift
    height_shift_range=0.15,  # Vertical shift
    shear_range=0.1,  # Shearing transformation
    zoom_range=0.15,  # Zoom in/out
    brightness_range=[0.8, 1.2],  # Brightness variation
    horizontal_flip=False,  # Don't flip (ASL is not symmetric)
    fill_mode='nearest',
    validation_split=0.2  # 80% train, 20% validation
)

# Validation data - ONLY rescaling (no augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

print("✓ Data augmentation configured")
print("  - Rotation: ±20°")
print("  - Shift: ±15%")
print("  - Zoom: ±15%")
print("  - Brightness: 80-120%")

## Cell 5: Load Training and Validation Data

In [ ]:
print("[2/5] Loading training and validation data...")

# Training data generator
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

# Validation data generator
val_generator = val_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

num_classes = len(train_generator.class_indices)

print(f"✓ Training samples: {train_generator.samples:,}")
print(f"✓ Validation samples: {val_generator.samples:,}")
print(f"✓ Number of classes: {num_classes}")
print(f"✓ Class names: {list(train_generator.class_indices.keys())}")

## Cell 6: Visualize Sample Augmented Images

In [ ]:
# Visualize augmented images
print("Visualizing augmented training samples...")

plt.figure(figsize=(15, 5))
sample_batch, sample_labels = next(train_generator)

for i in range(min(8, len(sample_batch))):
    plt.subplot(2, 4, i + 1)
    plt.imshow(sample_batch[i])
    class_name = list(train_generator.class_indices.keys())[int(sample_labels[i])]
    plt.title(f"Class: {class_name}")
    plt.axis('off')

plt.tight_layout()
plt.savefig('augmented_samples.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Sample images saved as 'augmented_samples.png'")

## Cell 7: Build Model with Strong Regularization

In [ ]:
print("[3/5] Building REGULARIZED model architecture...")

model = keras.Sequential([
    # Input layer
    keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    
    # Block 1 - with L2 regularization
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                       kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                       kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),
    
    # Block 2
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                       kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                       kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.4),
    
    # Block 3 - Deeper layer
    keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                       kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.5),
    
    # Dense layers with strong regularization
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu',
                      kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.6),
    
    keras.layers.Dense(128, activation='relu',
                      kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(num_classes, activation='softmax')
])

print("✓ Model architecture created")
model.summary()

## Cell 8: Compile Model

In [ ]:
print("[4/5] Compiling model...")

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✓ Model compiled successfully")

## Cell 9: Setup Callbacks (Anti-Overfitting)

In [ ]:
# Critical callbacks to prevent overfitting
callbacks = [
    # Save BEST model (not last)
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # EARLY STOPPING - stops when validation stops improving
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,  # Stop if no improvement for 5 epochs
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate when plateau
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=0.00001
    ),
]

print("✓ Callbacks configured:")
print("  - ModelCheckpoint: Save best model")
print("  - EarlyStopping: Patience = 5 epochs")
print("  - ReduceLROnPlateau: Patience = 3 epochs")

## Cell 10: Train Model

In [ ]:
print("[5/5] Training model...")
print(f"Max epochs: {EPOCHS} (early stopping enabled)\n")

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Training complete!")

## Cell 11: Plot Training History

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
ax1.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss plot
ax2.plot(history.history['loss'], label='Train Loss', linewidth=2)
ax2.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training history saved as 'training_history.png'")

## Cell 12: Evaluate Final Results

In [ ]:
# Calculate final metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
gap = final_train_acc - final_val_acc

print("="*60)
print("FINAL RESULTS:")
print("="*60)
print(f"Training Accuracy:   {final_train_acc*100:.2f}%")
print(f"Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"Training Loss:       {final_train_loss:.4f}")
print(f"Validation Loss:     {final_val_loss:.4f}")
print(f"\nOverfitting Gap:     {gap*100:.2f}%")

# Overfitting assessment
if gap < 0.05:
    print("✓ EXCELLENT: Low overfitting! Model generalizes well.")
elif gap < 0.10:
    print("⚠ ACCEPTABLE: Moderate overfitting. Consider more regularization.")
else:
    print("❌ WARNING: High overfitting! Increase dropout or get more data.")

print(f"\n✓ Best model saved as 'best_model.h5'")

## Cell 13: Convert to TFLite

In [ ]:
print("Converting model to TFLite...")

# Load best model
best_model = keras.models.load_model('best_model.h5')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Get file sizes
h5_size = os.path.getsize('best_model.h5') / (1024 * 1024)
tflite_size = os.path.getsize('model.tflite') / (1024 * 1024)

print(f"✓ TFLite model saved as 'model.tflite'")
print(f"  H5 size: {h5_size:.2f} MB")
print(f"  TFLite size: {tflite_size:.2f} MB")
print(f"  Compression: {(1 - tflite_size/h5_size)*100:.1f}%")

## Cell 14: Generate Labels File

In [ ]:
# Save class labels
class_labels = sorted(train_generator.class_indices.keys())

with open('labels.txt', 'w') as f:
    for label in class_labels:
        f.write(f"{label}\n")

print(f"✓ Labels saved to 'labels.txt'")
print(f"  Classes: {class_labels}")

## Cell 15: Summary and Next Steps

In [ ]:
print("="*60)
print("TRAINING COMPLETE!")
print("="*60)
print("\nFiles generated:")
print("  ✓ best_model.h5 - Best Keras model")
print("  ✓ model.tflite - Mobile-optimized model")
print("  ✓ labels.txt - Class labels")
print("  ✓ training_history.png - Training curves")
print("  ✓ augmented_samples.png - Sample images")
print("\nNext steps:")
print("  1. Copy model.tflite to Flutter assets/")
print("  2. Copy labels.txt to Flutter assets/")
print("  3. Test the model in your Flutter app")
print("  4. Monitor real-world performance")
print("\n🎉 Ready for deployment!")